In [1]:
# Importer les packages
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, GRU, Dropout
from sklearn.metrics import accuracy_score

# Importation des données

In [2]:
# Importer les données
path1 = './data/tweet_data.csv'
path2 = './data/Tweets_data.csv'

df_tweet = pd.read_csv(path1)
df_Tweets = pd.read_csv(path2)

# Afficher les données
print('tweet df')
print(df_tweet.head())

print('\n------è-------\n')

print('Tweets df')
print(df_Tweets.head())

tweet df
   label                                              tweet
0      1  @user homegrown rightie white americans are 10...
1      1  @user @user  is this a new one? either way, #m...
2      0  i am thankful for pillows. #thankful #positive...
3      0  it's unfounate today that aids is under contro...
4      0      i am super. #i_am #positive #affirmation     

------è-------

Tweets df
  coachella_sentiment                                               text
0            positive  #Coachella2015 tickets selling out in less tha...
1            positive  RT @sudsybuddy: WAIT THIS IS ABSOLUTE FIRE _ÙÓ...
2            positive  #Coachella2015 #VIP passes secured! See you th...
3            positive  PhillyÛªs @warondrugsjams will play #Coachell...
4            positive  If briana and her mom out to #Coachella2015  i...


# Concaténation des deux tweet_data dataframe

In [3]:
# Uniformisons les labels en (label et tweet) ...
df_Tweets.columns = ['label', 'tweet']

In [4]:
# selectionnons uniquement les lignes positive et négative de df_Tweets ...
data = []
for i in range(df_Tweets.shape[0]):
    label = df_Tweets["label"][i]
    tweet = df_Tweets["tweet"][i]
    
    if(label == "positive" or label == "negative"):
        label = 0 if label == "positive" else 1
        data.append({'label': label, 'tweet': tweet})
    
df_Tweets = pd.DataFrame(data)

In [5]:
# Affichons de nouveau les données des deux df
print('tweet df')
print(df_tweet.head())

print('\n------è-------\n')

print('Tweets df')
print(df_Tweets.head())

tweet df
   label                                              tweet
0      1  @user homegrown rightie white americans are 10...
1      1  @user @user  is this a new one? either way, #m...
2      0  i am thankful for pillows. #thankful #positive...
3      0  it's unfounate today that aids is under contro...
4      0      i am super. #i_am #positive #affirmation     

------è-------

Tweets df
   label                                              tweet
0      0  #Coachella2015 tickets selling out in less tha...
1      0  RT @sudsybuddy: WAIT THIS IS ABSOLUTE FIRE _ÙÓ...
2      0  #Coachella2015 #VIP passes secured! See you th...
3      0  PhillyÛªs @warondrugsjams will play #Coachell...
4      0  If briana and her mom out to #Coachella2015  i...


In [6]:
# Concaténons les deux data frame en un seul ...
df = pd.concat([df_tweet, df_Tweets])
df.reset_index(drop = True, inplace = True)

In [7]:
print(df['tweet'][4484])

#Coachella2015 tickets selling out in less than 40 minutes _Ù_¦_Ù___Ù___Ù÷_ÙÎµ_ÙÎµ_Ù___Ù_¦ http://t.co/SmoXyteIMJ


# Nettoyage du df

In [8]:
!pip install contractions


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


- Nettoyage des données

In [9]:
import contractions
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
Stopwords = stopwords.words('english')

def cleannig_tweet(tweet):
    expanded_all = []
    tweet = re.sub(r'http[s]*:?//\S+','', tweet)
    tweet = re.sub(r'@[\w\-\.]+', '', tweet)
    tweet = re.sub(r'[\w\-\.]+@[\w\-\.]+', '', tweet)
    tweet = re.sub(r'&\w+','', tweet)
    tweet = re.sub(r'[^a-zA-Z\s]+', ' ', tweet)
    tweet = re.sub(r'^\s|\s$', '', tweet)
    tweet = re.sub(r'\s{2,}', ' ', tweet).lower()
    
    
    for word in tweet.split():
        expanded_all.append(contractions.fix(word))
    tweet = ' '.join(expanded_all)
    
    
    tweet = ' '.join([word for word in tweet.split() if word not in Stopwords])

    tweet = [lemma.lemmatize(word) for word in tweet.split()]
    tweet = " ".join(tweet)
    return tweet

In [10]:
# Appliquer la fonction cleannig_tweet
df['new_tweet'] = df.tweet.apply(func = cleannig_tweet)

In [11]:
print(df['tweet'][4484])
print(df['new_tweet'][4484])

#Coachella2015 tickets selling out in less than 40 minutes _Ù_¦_Ù___Ù___Ù÷_ÙÎµ_ÙÎµ_Ù___Ù_¦ http://t.co/SmoXyteIMJ
coachella ticket selling le minute


In [12]:
# Affichons le jeu de données
df.head()

,label,tweet,new_tweet
0,1,@user homegrown rightie white americans are 10...,homegrown rightie white american time likely h...
1,1,"@user @user is this a new one? either way, #m...",new one either way men disgraced harassment ve...
2,0,i am thankful for pillows. #thankful #positive...,thankful pillow thankful positive
3,0,it's unfounate today that aids is under contro...,unfounate today aid control hate kill gay year...
4,0,i am super. #i_am #positive #affirmation,super positive affirmation


- Vectorisation des tweets

In [14]:
# # Vectorison avec TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Initialisation
# vect = TfidfVectorizer()

# # Matrice des tweets
# vect_matrix = vect.fit_transform(df['new_tweet'])

# vect_array = vect_matrix.toarray()

# nbr_word_unique = len(vect.get_feature_names_out())

In [13]:
# Vectorison avec Tokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Nombre de mots
nbr_word_unique = len(set(" ".join(df.new_tweet).split()))

# Initialiser le modèle Tokenizer
tokenizer = Tokenizer(num_words = nbr_word_unique, split=' ')

# Entrainer les données
tokenizer.fit_on_texts(df['new_tweet'].values)

# Vectoriser
vect_array = tokenizer.texts_to_sequences(df['new_tweet'].values)

# Padding
vect_array = pad_sequences(vect_array)

In [14]:
# Sauvegarder le tokenizer en JSON
tokenizer_json = tokenizer.to_json()
with open('english_tokenizer.json', 'w', encoding='utf-8') as json_file:
    json_file.write(tokenizer_json)

In [15]:
# Le nombre de mot uniques dans les tweets
nbr_word_unique

11229

# Définission des modèles

In [28]:
# Installons keras-tuner
!pip install -q -U keras-tuner


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


- Séparation des données en train, test et validation

In [16]:
# Importer train_test_split
from sklearn.model_selection import train_test_split

# Labels
y = df.label.values

# Splitter en train et test
x_train, x_test, y_train, y_test = train_test_split(vect_array, y, test_size=0.2, random_state=42)

# Splitter validation et test
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [17]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [18]:
# Définissons l'opitmizer
from keras.optimizers import Adam

In [19]:
def bidir_lstm_model_builder(hp):
    emb_dimention = hp.Int('emb_dimention', min_value=32, max_value=512, step=32)
    nbr_neurones = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    nbr_neurones_l2 = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    dropout = hp.Boolean("dropout")
    
    Adam(learning_rate=learning_rate)
    
    bidir_lstm_model=Sequential()
    bidir_lstm_model.add(Embedding(nbr_word_unique, emb_dimention, input_length=vect_array.shape[1]))
    bidir_lstm_model.add(Bidirectional(LSTM(nbr_neurones, return_sequences=True)))
    if dropout :
        bidir_lstm_model.add(Dropout(0.4))
    bidir_lstm_model.add(Bidirectional(LSTM(nbr_neurones_l2)))
    if dropout: 
        bidir_lstm_model.add(Dropout(0.4))
    bidir_lstm_model.add(Dense(1,activation='sigmoid'))
    bidir_lstm_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return bidir_lstm_model

In [20]:
def bidir_gru_model_builder(hp):
    emb_dimention = hp.Int('emb_dimention', min_value=32, max_value=512, step=32)
    nbr_neurones = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    nbr_neurones_l2 = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    dropout = hp.Boolean("dropout")
    
    Adam(learning_rate=learning_rate)
    
    bidir_gru_model=Sequential()
    bidir_gru_model.add(Embedding(nbr_word_unique, emb_dimention, input_length=vect_array.shape[1]))
    bidir_gru_model.add(Bidirectional(GRU(nbr_neurones, return_sequences=True)))
    if dropout :
        bidir_gru_model.add(Dropout(0.4))
    bidir_gru_model.add(Bidirectional(GRU(nbr_neurones_l2)))
    if dropout: 
        bidir_gru_model.add(Dropout(0.4))
    bidir_gru_model.add(Dense(1,activation='sigmoid'))
    bidir_gru_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return bidir_gru_model

In [21]:
def lstm_model_builder(hp):
    emb_dimention = hp.Int('emb_dimention', min_value=32, max_value=512, step=32)
    nbr_neurones = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    nbr_neurones_l2 = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    dropout = hp.Boolean("dropout")
    
    Adam(learning_rate=learning_rate)
    
    lstm_model=Sequential()
    lstm_model.add(Embedding(nbr_word_unique, emb_dimention, input_length=vect_array.shape[1]))
    lstm_model.add(LSTM(nbr_neurones, return_sequences=True))
    if dropout :
        lstm_model.add(Dropout(0.4))
    lstm_model.add(LSTM(nbr_neurones_l2))
    if dropout: 
        lstm_model.add(Dropout(0.4))
    lstm_model.add(Dense(1,activation='sigmoid'))
    lstm_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return lstm_model

In [22]:
def gru_model_builder(hp):
    emb_dimention = hp.Int('emb_dimention', min_value=32, max_value=512, step=32)
    nbr_neurones = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    nbr_neurones_l2 = hp.Int('nbr_neurones', min_value=32, max_value=512, step=32)
    learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    dropout = hp.Boolean("dropout")
    
    Adam(learning_rate=learning_rate)
    
    gru_model=Sequential()
    gru_model.add(Embedding(nbr_word_unique, emb_dimention, input_length=vect_array.shape[1]))
    gru_model.add(GRU(nbr_neurones, return_sequences=True))
    if dropout :
        gru_model.add(Dropout(0.4))
    gru_model.add(GRU(nbr_neurones_l2))
    if dropout: 
        gru_model.add(Dropout(0.4))
    gru_model.add(Dense(1,activation='sigmoid'))
    gru_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return gru_model

In [23]:
def tune_model(model_to_tune):
    # Mettons en place un rapel d'arrêt après avoir atteint une certaine valeur
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    
    # Instancions le tuner
    tuner = kt.RandomSearch(
        model_to_tune,
        objective='val_accuracy',
        max_trials = 2,
        overwrite = True,
        directory='tuners_dir',
        project_name='intro_to_kt'
    )
    
    # Exécutons la recherche des paramètres
    tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=3, callbacks=[stop_early])
    
    return tuner

In [24]:
# Récupérons les models tunes
bidir_lstm_tuner = tune_model(bidir_lstm_model_builder)
bidir_gru_tuner = tune_model(bidir_gru_model_builder)
lstm_tuner = tune_model(lstm_model_builder)
gru_tuner = tune_model(gru_model_builder)

Trial 2 Complete [00h 04m 25s]
val_accuracy: 0.8483606576919556

Best val_accuracy So Far: 0.8483606576919556
Total elapsed time: 00h 07m 34s


In [25]:
# Construisons les modèles
bidir_lstm_model = bidir_lstm_tuner.hypermodel.build(bidir_lstm_tuner.get_best_hyperparameters(num_trials=1)[0])

bidir_gru_model = bidir_gru_tuner.hypermodel.build(bidir_gru_tuner.get_best_hyperparameters(num_trials=1)[0])

lstm_model = lstm_tuner.hypermodel.build(lstm_tuner.get_best_hyperparameters(num_trials=1)[0])

gru_model = gru_tuner.hypermodel.build(gru_tuner.get_best_hyperparameters(num_trials=1)[0])

In [26]:
models = [
    {"name": "bidirectional LSTM", "model": bidir_lstm_model, "score": 0},
    {"name": "LSTM", "model": lstm_model, "score": 0},
    {"name": "bidirectional GRU", "model": bidir_gru_model, "score": 0},
    {"name": "GRU", "model": gru_model, "score": 0}
]

In [27]:
for model in models:
    print(model["name"])
    print(model["model"].summary())
    print('\n\n')

bidirectional LSTM
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 21, 224)           2515296   
                                                                 
 bidirectional (Bidirection  (None, 21, 960)           2707200   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 21, 960)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 960)               5533440   
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 960)               0         
                                                                 
 dense_1 (Dense)             (None,

# Entrainement et Validation des modèles 

- Entrainement et validation

In [28]:
# entrainement et validation des modèles
for model in models:
    print(f"Modèle {model['name']}")
    model['model'].fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=64)
    
    # Prédire les labels de x_val
    y_pred = model['model'].predict(x_val)
    
    # Arrondir les valeurs
    y_pred = np.round(y_pred)
    
    # calculons l'accuracy
    model['score'] = accuracy_score(y_pred, y_val)

Modèle bidirectional LSTM
Epoch 1/5
92/92 [==============================] - 219s 2s/step - loss: 0.4757 - accuracy: 0.7758 - val_loss: 0.4009 - val_accuracy: 0.8251
Epoch 2/5
92/92 [==============================] - 199s 2s/step - loss: 0.2122 - accuracy: 0.9206 - val_loss: 0.4189 - val_accuracy: 0.8224
Epoch 3/5
92/92 [==============================] - 212s 2s/step - loss: 0.1105 - accuracy: 0.9617 - val_loss: 0.5576 - val_accuracy: 0.8128
Epoch 4/5
92/92 [==============================] - 212s 2s/step - loss: 0.0878 - accuracy: 0.9710 - val_loss: 0.7291 - val_accuracy: 0.8074
Epoch 5/5
23/23 [==============================] - 15s 536ms/step
Modèle LSTM
Epoch 1/5
92/92 [==============================] - 85s 853ms/step - loss: 0.4986 - accuracy: 0.7602 - val_loss: 0.3738 - val_accuracy: 0.8333
Epoch 2/5
92/92 [==============================] - 69s 747ms/step - loss: 0.2224 - accuracy: 0.9155 - val_loss: 0.4094 - val_accuracy: 0.8210
Epoch 3/5
92/92 [==============================] - 7

In [29]:
# Affichons les scores et choisissons le meilleur modèle
choosed_model = models[0]
for model in models:
    print(f"{model['name']}: {model['score']}")
    if(model['score'] > choosed_model['score']):
        choosed_model = model

bidirectional LSTM: 0.7909836065573771
LSTM: 0.7937158469945356
bidirectional GRU: 0.8169398907103825
GRU: 0.8046448087431693


In [30]:
# Affichons le modèle choisi
print("Le modèle choisi est :")
print(f"{choosed_model['name']}: {choosed_model['score']}")

Le modèle choisi est :
bidirectional GRU: 0.8169398907103825


In [31]:
# sauvegarde du modèle
choosed_model['model'].save("models/english_sentiment_analysis.h5")

c:\Users\morcodou.seck\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
